### FAISS DB

We can Store Vectors in FAISS DB 
We can calculate the similarity between documents.

And we can have Vector store as retriever and use in the RAG Pipeline

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/I572648/Library/CloudStorage/OneDrive-SAPSE/Desktop/Git/GenAI/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
vector_query = embeddings.embed_query("This is a test query")
len(vector_query)  # Should return the length of the embedding vector

384

In [3]:
# Cosine similarity
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
documents = ["What is the capital of USA?",
             "Who is the president of USA?",
             "who is the prime minister of INDIA?"]

doc_embed = embeddings.embed_documents(documents)

In [5]:
my_query = "Naredra Modi is the prime minister of India"
query_embed = embeddings.embed_query(my_query)

# Cosine similarity between query and documents
similarity_scores = cosine_similarity([query_embed], doc_embed)
similarity_scores

array([[0.04977604, 0.2418052 , 0.66529683]])

In [6]:
# Euclidean distance
from sklearn.metrics.pairwise import euclidean_distances
euclidean_scores = euclidean_distances([query_embed], doc_embed)
euclidean_scores

array([[1.37856736, 1.23141773, 0.81817257]])

### Vector store using FAISS

In [14]:
# Create a vectorstore using FAISS
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

index = faiss.IndexFlatL2(384) # 384 is the dimension of the embedding vector

vectorstore = FAISS(embedding_function=embeddings, index=index, docstore=InMemoryDocstore({}), index_to_docstore_id={})
# Add documents to the vectorstore
vectorstore.add_texts(texts=["AI is future","AI is the future of technology", "AI is the future of work", "Dogs are bark"])

['07f57caf-53b4-4022-a204-6b7c0228a2f7',
 'fc662d75-5418-48a3-8d45-13c794852ba3',
 '6ee5338b-942c-47a0-af85-6c17526cbf51',
 'be366424-4f8c-4564-9ed5-fbb59d673ba7']

In [15]:
vectorstore.similarity_search("What is the future of AI?", k=2)

[Document(id='fc662d75-5418-48a3-8d45-13c794852ba3', metadata={}, page_content='AI is the future of technology'),
 Document(id='07f57caf-53b4-4022-a204-6b7c0228a2f7', metadata={}, page_content='AI is future')]

In [16]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [17]:
index=faiss.IndexFlatIP(384) # for cosine similarity, use IndexFlatIP
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [18]:
vector_store.add_documents(documents=documents)

['f37fe5bd-2326-4d63-ad1c-6d302527b428',
 'e46086ff-8f32-4a1c-959e-b6db6b82e6b1',
 '3ab37bf0-dcdd-460b-ad44-b4263e5d5a81',
 '20833fb6-a630-47ff-9ed3-66403a5ad4f5',
 '86312460-bb52-4e20-bc54-6a6f899948a4',
 '78447514-10a9-4483-b707-c21b86358f8e',
 'c5672130-f691-4cbf-bfd6-5cfb2bae6ea7',
 'b086a16d-0ebc-4058-9080-f89d8109d588',
 'b5cae258-57c6-4a43-9512-1ce30dab2688',
 '6e51cafa-baa8-4613-b434-1b7f82e16d12']

In [19]:
vector_store.similarity_search("LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
)

[Document(id='3ab37bf0-dcdd-460b-ad44-b4263e5d5a81', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='b086a16d-0ebc-4058-9080-f89d8109d588', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [20]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
)

[Document(id='3ab37bf0-dcdd-460b-ad44-b4263e5d5a81', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='b086a16d-0ebc-4058-9080-f89d8109d588', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='6e51cafa-baa8-4613-b434-1b7f82e16d12', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='f37fe5bd-2326-4d63-ad1c-6d302527b428', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [21]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
)
result

[Document(id='20833fb6-a630-47ff-9ed3-66403a5ad4f5', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='e46086ff-8f32-4a1c-959e-b6db6b82e6b1', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='b5cae258-57c6-4a43-9512-1ce30dab2688', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [22]:
print(result[0].page_content)
result[0].metadata

Robbers broke into the city bank and stole $1 million in cash.


{'source': 'news'}

### Vector store as Retriever

In [23]:
# retrieve documents using retriever, we can use retriver instead of vector_store in RAG PIPELINE
retriver = vector_store.as_retriever(search_kwargs={"k":3})

In [24]:
retriver.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='3ab37bf0-dcdd-460b-ad44-b4263e5d5a81', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='b086a16d-0ebc-4058-9080-f89d8109d588', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='6e51cafa-baa8-4613-b434-1b7f82e16d12', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

### Save the Vectore store in local

In [25]:
vector_store.save_local("todays faiss index")

In [26]:
new_vector_store = FAISS.load_local("todays faiss index", embeddings, allow_dangerous_deserialization=True)
# Verify that the new vector store is loaded correctly
new_vector_store.similarity_search("LangChain provides abstractions to make working with LLMs easy", k=5)

[Document(id='3ab37bf0-dcdd-460b-ad44-b4263e5d5a81', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='b086a16d-0ebc-4058-9080-f89d8109d588', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='6e51cafa-baa8-4613-b434-1b7f82e16d12', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='20833fb6-a630-47ff-9ed3-66403a5ad4f5', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='c5672130-f691-4cbf-bfd6-5cfb2bae6ea7', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]